In [1]:
(version)

"7.7"

In [2]:
#lang iracket/lang #:require bhdl/splicing #:reader bhdl/splicing

In [3]:
(list (list 1 2 3) ..)

'(1 2 3)

In [4]:
(putenv "BHDL_LIBRARY_PATH" "~/bhdl/bhdl-footprints")
(require bhdl)

DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  ARC


In [5]:
(require racket/list
         racket/format
         (prefix-in pict: pict))

In [6]:
(define global
  (make-circuit
   #:external-pins (GND 3V3 5V VCC USB5V)))

# The placement engine

In [7]:
;; stop any existing placement engines
(stop-placement-engine)

#t

In [8]:
;; allow some times for the julia process to warm up
(start-placement-engine)

DEBUG:  Instantiating the julia project packages ..


Precompiling project...


DEBUG:  Starting the placement engine asynchronizely ..
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: running
The TCP port is still #f , wainting for 1 sec .. process status: run

In [10]:
(status-placement-engine)

running


In [11]:
;; For now, go to ~/bhdl/output.txt for output
;; (output-placement-engine)

# Keyboard Matrix

In [12]:
;; A small circuit with one key and one diode
(define (key-with-diode)
  (make-circuit 
   #:external-pins (left right)
   #:vars ([d (1N4148W)]
           [key (kailh-socket 1)])
   #:connect (*- self.left key d self.right)
   #:layout (vc-append key d)))

In [13]:
(show-layout (key-with-diode))

"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757211604675721730"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757211604675721754"


(pict '(picture 70.71060052116022 91.95262301108961 (put 0.0 27.709194312276576 #1=(prog #<procedure:...rivate/utils.rkt:1274:8> 64.24342869881303)) (put 25.290407775167797 0 #2=(prog #<procedure:...rivate/utils.rkt:1274:8> 7.709194312276576))) 70.71060052116022 91.95262301108961 32.12171434940652 3.854597156138288 (list (child (pict #1# 70.71060052116022 64.24342869881303 32.12171434940652 32.12171434940652 '() (bbox 0.0 0.0 70.71060052116022 64.24342869881303 32.12171434940652 32.12171434940652) #f) 0.0 27.709194312276576 1 1 0 0) (child #0=(pict #2# 20.129784970824623 7.709194312276576 3.854597156138288 3.854597156138288 '() (bbox 0.0 0.0 20.129784970824623 7.709194312276576 3.854597156138288 3.854597156138288) #f) 25.290407775167797 0 1 1 0 0)) #f #0#)

In [14]:
;; create a 4x5 matrix
(define matrix
  (for/list ([row (in-range 4)])
            (for/list ([col (in-range 5)])
                      (key-with-diode))))

In [15]:
;; connect the matrix
(define matrix-module
  (make-circuit 
   #:external-pins (row[4] col[5])
   ;; col in, row out
   #:connect (for*/list ([i (in-range 4)]
                        [j (in-range 5)])
                       (debug "connecting" i j)
                       (*- self.col[j]
                           (list-ref (list-ref matrix i) j)
                           self.row[i]))
   #:layout (inset (vc-append
             (for/list ([row matrix])
                      (hc-append row ..)
                      ) ..)
                   20)))

DEBUG:  connecting 0 0
DEBUG:  connecting 0 1
DEBUG:  connecting 0 2
DEBUG:  connecting 0 3
DEBUG:  connecting 0 4
DEBUG:  connecting 1 0
DEBUG:  connecting 1 1
DEBUG:  connecting 1 2
DEBUG:  connecting 1 3
DEBUG:  connecting 1 4
DEBUG:  connecting 2 0
DEBUG:  connecting 2 1
DEBUG:  connecting 2 2
DEBUG:  connecting 2 3
DEBUG:  connecting 2 4
DEBUG:  connecting 3 0
DEBUG:  connecting 3 1
DEBUG:  connecting 3 2
DEBUG:  connecting 3 3
DEBUG:  connecting 3 4


In [16]:
(parameterize ([current-directory "./out/demo-board/matrix"]
               [padding-general 2])
    (circuit-export matrix-module
                    #:auto-place #f
                    #:formats '(kicad pdf png svg)))

generating KiCAD PCB ..
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757221604675722093"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757221604675722101"
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/matrix/out.kicad_pcb
DEBUG:  Number of conflicts: 0
generating pdf ..
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/matrix/out.pdf
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/matrix/out.png
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/matrix/out.svg


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 467.81049204435845) 473.5530026058011 467.81049204435845 467.81049204435845 0.0 (list (child (pict '(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #1071=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1067=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1065=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #1053=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1049=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1047=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #1035=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1031=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1029=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #1017=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1013=(picture 473.5530026058011 467.81049204435845 (put 0 0 #1011=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #999=(picture 473.5530026058011 467.81049204435845 (put 0 0 #995=(picture 473.5530026058011 467.81049204435845 (put 0 0 #993=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #981=(picture 473.5530026058011 467.81049204435845 (put 0 0 #977=(picture 473.5530026058011 467.81049204435845 (put 0 0 #975=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #963=(picture 473.5530026058011 467.81049204435845 (put 0 0 #959=(picture 473.5530026058011 467.81049204435845 (put 0 0 #957=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #945=(picture 473.5530026058011 467.81049204435845 (put 0 0 #941=(picture 473.5530026058011 467.81049204435845 (put 0 0 #939=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #927=(picture 473.5530026058011 467.81049204435845 (put 0 0 #923=(picture 473.5530026058011 467.81049204435845 (put 0 0 #921=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #909=(picture 473.5530026058011 467.81049204435845 (put 0 0 #905=(picture 473.5530026058011 467.81049204435845 (put 0 0 #903=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #891=(picture 473.5530026058011 467.81049204435845 (put 0 0 #887=(picture 473.5530026058011 467.81049204435845 (put 0 0 #885=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #873=(picture 473.5530026058011 467.81049204435845 (put 0 0 #869=(picture 473.5530026058011 467.81049204435845 (put 0 0 #867=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #855=(picture 473.5530026058011 467.81049204435845 (put 0 0 #851=(picture 473.5530026058011 467.81049204435845 (put 0 0 #849=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #837=(picture 473.5530026058011 467.81049204435845 (put 0 0 #833=(picture 473.5530026058011 467.81049204435845 (put 0 0 #831=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #819=(picture 473.5530026058011 467.81049204435845 (put 0 0 #815=(picture 473.5530026058011 467.81049204435845 (put 0 0 #813=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #801=(picture 473.5530026058011 467.81049204435845 (put 0 0 #797=(picture 473.5530026058011 467.81049204435845 (put 0 0 #795=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #783=(picture 473.5530026058011 467.81049204435845 (put 0 0 #779=(picture 473.5530026058011 467.81049204435845 (put 0 0 #777=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #765=(picture 473.5530026058011 467.81049204435845 (put 0 0 #761=(picture 473.5530026058011 467.81049204435845 (put 0 0 #759=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #747=(picture 473.5530026058011 467.81049204435845 (put 0 0 #743=(picture 473.5530026058011 467.81049204435845 (put 0 0 #741=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #729=(picture 473.5530026058011 467.81049204435845 (put 0 0 #725=(picture 473.5530026058011 467.81049204435845 (put 0 0 #723=(picture 473.5530026058011 467.81049204435845 (put 0.0 0.0 #711=(picture 473.5530026058011 467.81049204435845 (put 0 0 #707=(picture 473.5530026058011 467.

# MCU Board

In [17]:
(define (icsp-header)
   (make-circuit #:vars ([h (PinHeader2 3)])
                   #:external-pins (MOSI SCK MISO RESET VCC GND)
                   #:connect (*= (self (MISO VCC SCK MOSI RESET GND))
                                 (h [1 2 3 4 5 6]))
                   #:layout h))

In [18]:
;; Now using the Atmega32u4 from the arduino board
(define mcu-module 
  (make-circuit 
 #:vars ([mcu (ATmega32U4 #:FP "TQFP-44")]
         [usb (USB-Micro)]
         [icsp (icsp-header)]
         [xtal (Crystal-4 '16MHz)])
 #:external-pins (row[4] col[5])
 #:layout (vc-append 50 (rotate usb pi) mcu icsp)
 ;; define what are the row and col scan pins
 #:connect (*= (self row [0 3])
               (self row [0 3])
               (mcu [PB0 PB4 PB5 PB6]))
 #:connect (*= (self col [0 4])
                 (mcu [PD0 PD1 PD2 PD3 PD4]))
   ;; use a Pin for blinking LED
   #:connect (*- mcu.PC6 (R '1k) (LED0603 'red) global.GND)
                    
 ;; connect the ICSP header
 #:connect (*= (mcu (MOSI SCLK MISO RESET VCC GND))
               (icsp (MOSI SCK MISO RESET VCC GND)))
 #:connect
 (list (*- mcu.VCC mcu.UVCC mcu.AVCC global.VCC)
         (*- mcu.GND mcu.UGND global.GND)
         ;; reset circuit and switch
         (*- mcu.RESET (*< (LL4148)
                           (R '10k)) global.5V)
       ;; FIXME [-1] because (Switch) is parsed as the index otherwise
         (*- mcu.RESET[-1] (Switch) global.GND)
       ;; capacitors
       (*- mcu.UCAP (C '1uf) global.GND)
       (*- global.VCC (*< (C '1uf)
                          (C '100nf)
                          (C '100nf)
                          (C '100nf)
                          (C '100nf))
           global.GND)
         ;; crystal
       (*- mcu.XTAL1 xtal.XIN)
       (*- xtal.XOUT mcu.XTAL2)
       (*- xtal.GND global.GND)
         (*- mcu.XTAL1 (C '22pf) global.GND)
         (*- mcu.XTAL2 (C '22pf) global.GND)
       ;; USB
       (*- mcu.D+ (R 22) usb.D+)
       (*- mcu.D- (R 22) usb.D-)
       ;; just use USB for supplying power
       (*- mcu.VBUS usb.VBUS global.VCC)
       (*- mcu.UGND usb.GND global.GND)
       )))

"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722753"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722767"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722784"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722793"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722800"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722806"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722812"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722815"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046757221604675722819"


In [19]:
;; connect MCU
(define mcu-board
  (make-circuit 
   #:layout (inset (cb-superimpose mcu-module) 50 0)
   #:connect mcu-module))

In [20]:
(parameterize ([current-directory "./out/demo-board/mcu-board"]
               [padding-general 2])
    (circuit-export mcu-board
                    #:auto-place #t
                    #:formats '(kicad pdf bom png)))

DEBUG:  sending for placement ..
DEBUG:  saving ..
generating KiCAD PCB ..
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756322"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756375"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756395"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756408"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756420"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756429"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756439"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756442"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046757561604675756449"
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/mcu-board/out.kicad_pcb
Generating BOM ..
DEBUG:  Number 

(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 220.56350119143912) 157.39079630427298 220.56350119143912 220.56350119143912 0.0 (list (child (pict '(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #850=(picture 157.39079630427298 220.56350119143912 (put 0 0 #846=(picture 157.39079630427298 220.56350119143912 (put 0 0 #844=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #832=(picture 157.39079630427298 220.56350119143912 (put 0 0 #828=(picture 157.39079630427298 220.56350119143912 (put 0 0 #826=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #814=(picture 157.39079630427298 220.56350119143912 (put 0 0 #810=(picture 157.39079630427298 220.56350119143912 (put 0 0 #808=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #796=(picture 157.39079630427298 220.56350119143912 (put 0 0 #792=(picture 157.39079630427298 220.56350119143912 (put 0 0 #790=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #778=(picture 157.39079630427298 220.56350119143912 (put 0 0 #774=(picture 157.39079630427298 220.56350119143912 (put 0 0 #772=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #760=(picture 157.39079630427298 220.56350119143912 (put 0 0 #756=(picture 157.39079630427298 220.56350119143912 (put 0 0 #754=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #742=(picture 157.39079630427298 220.56350119143912 (put 0 0 #738=(picture 157.39079630427298 220.56350119143912 (put 0 0 #736=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #724=(picture 157.39079630427298 220.56350119143912 (put 0 0 #720=(picture 157.39079630427298 220.56350119143912 (put 0 0 #718=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #706=(picture 157.39079630427298 220.56350119143912 (put 0 0 #702=(picture 157.39079630427298 220.56350119143912 (put 0 0 #700=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #688=(picture 157.39079630427298 220.56350119143912 (put 0 0 #684=(picture 157.39079630427298 220.56350119143912 (put 0 0 #682=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #670=(picture 157.39079630427298 220.56350119143912 (put 0 0 #666=(picture 157.39079630427298 220.56350119143912 (put 0 0 #664=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #652=(picture 157.39079630427298 220.56350119143912 (put 0 0 #648=(picture 157.39079630427298 220.56350119143912 (put 0 0 #646=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #634=(picture 157.39079630427298 220.56350119143912 (put 0 0 #630=(picture 157.39079630427298 220.56350119143912 (put 0 0 #628=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #616=(picture 157.39079630427298 220.56350119143912 (put 0 0 #612=(picture 157.39079630427298 220.56350119143912 (put 0 0 #610=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #598=(picture 157.39079630427298 220.56350119143912 (put 0 0 #594=(picture 157.39079630427298 220.56350119143912 (put 0 0 #592=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #580=(picture 157.39079630427298 220.56350119143912 (put 0 0 #576=(picture 157.39079630427298 220.56350119143912 (put 0 0 #574=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #562=(picture 157.39079630427298 220.56350119143912 (put 0 0 #558=(picture 157.39079630427298 220.56350119143912 (put 0 0 #556=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #544=(picture 157.39079630427298 220.56350119143912 (put 0 0 #540=(picture 157.39079630427298 220.56350119143912 (put 0 0 #538=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #526=(picture 157.39079630427298 220.56350119143912 (put 0 0 #522=(picture 157.39079630427298 220.56350119143912 (put 0 0 #520=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #508=(picture 157.39079630427298 220.56350119143912 (put 0 0 #504=(picture 157.39079630427298 220.56350119143912 (put 0 0 #502=(picture 157.39079630427298 220.56350119143912 (put 0.0 0.0 #490=(picture 157.39079630427298 220.563501191

# Integrate MCU and Key matrix

In [21]:
(define keyboard (make-circuit
 #:connect (*= (mcu-module row [0 3])
               (matrix-module row [0 3]))
 #:connect (*= (mcu-module col [0 4])
               (matrix-module col [0 4]))
 #:layout (vc-append mcu-module matrix-module)
                  ))

In [22]:
(parameterize ([current-directory "./out/demo-board/keyboard"]
                 [padding-general 2])
                (circuit-export keyboard 
                                #:auto-place #t
                                #:formats '(kicad pdf png bom)))

DEBUG:  sending for placement ..
DEBUG:  saving ..
generating KiCAD PCB ..
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/keyboard/out.kicad_pcb
Generating BOM ..
DEBUG:  Number of conflicts: 0
generating pdf ..
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/keyboard/out.pdf
link: /home/jovyan/bhdl/bhdl-test/out/demo-board/keyboard/out.png


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 480.0) 320.88337999602567 480.0 480.0 0.0 (list (child (pict '(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #2094=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2090=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2088=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #2076=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2072=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2070=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #2058=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2054=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2052=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #2040=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2036=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2034=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #2022=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2018=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2016=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #2004=(picture 473.5530026058011 708.3739932357976 (put 0 0 #2000=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1998=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1986=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1982=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1980=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1968=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1964=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1962=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1950=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1946=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1944=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1932=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1928=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1926=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1914=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1910=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1908=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1896=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1892=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1890=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1878=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1874=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1872=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1860=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1856=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1854=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1842=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1838=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1836=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1824=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1820=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1818=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1806=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1802=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1800=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1788=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1784=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1782=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1770=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1766=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1764=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1752=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1748=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1746=(picture 473.5530026058011 708.3739932357976 (put 0.0 0.0 #1734=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1730=(picture 473.5530026058011 708.3739932357976 (put 0 0 #1728=(picture 473.55300260